# Importação das Bibliotecas que serão utilizadas no Modelo

In [ ]:
!pip install --upgrade tensorflow-addons

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import gdown
import plotly.express as px
import plotly.graph_objects as go
from tensorflow.keras import backend as K

## Download dos arquivos contendo os datasets de consumo desde 2019 a 2024
  - Aqui retiramos a base de 2020, por conta da pandemia do Coronavírus. Foi uma escolha do grupo devido à possibilidade de discrepância nas leituras

In [ ]:
arquivo_destino_base = "dataset_{}.csv"

ids = {
    "consumo_2024": "1-iXT7eaJWQokHf9cyfrB8N5wvkdhgjJW",
    "consumo_2023": "1-WfvkRwaRr85B_Joxcm9xVdpyg5NBAmp",
    "consumo_2022": "1-Uu4Tf4lufJVFeJnYKc5w7OeW66pe1eC",
    "consumo_2021": "1-2PsTLzG4dcY4wM0p7vFfabUuLv950gC",
    "consumo_2020": "1-1pOoa0eJlNJ94BMi7p4PTx5KUS96mhX",
    "consumo_2019": "1-2PsTLzG4dcY4wM0p7vFfabUuLv950gC",
    "CONSUMO_GERAL": "1-IOqfwmh_tTIDHeOer8J-HkGFtwuX67g",
}


dataframes = {}


for key, file_id in ids.items():
    url = f"https://drive.google.com/uc?id={file_id}"
    arquivo_destino = arquivo_destino_base.format(key)

    gdown.download(url, arquivo_destino, quiet=False)

In [ ]:
arquivos_csv = [
    "./dataset_consumo_2024.csv",
    "./dataset_consumo_2023.csv",
    "./dataset_consumo_2022.csv",
    "./dataset_consumo_2021.csv",
    "./dataset_consumo_2019.csv",
]

ALL_COLUMNS_CONSUMO_GERAL = pd.concat([pd.read_csv(arquivo, delimiter=";") for arquivo in arquivos_csv], axis=0)

In [ ]:
consumo_geral = pd.read_csv('/content/dataset_CONSUMO_GERAL.csv')

## Download do dataset com o Target das Fraudes

In [ ]:
file_id_fraudes = "1-MbIlChqQapcxFkoJgpbQIsN9FBLfbX1"
url_fraudes = f"https://drive.google.com/uc?id={file_id_fraudes}"

gdown.download(url_fraudes, quiet=False)

In [ ]:
fraudes = pd.read_csv('/content/fraudes.csv')

### A tabela "ALL_COLUMNS_CONSUMO_GERAL" possui todas as tabelas de consumo e a partir disso decidimos considerar algumas colunas categóricas que podem ajudar a melhorar o desempenho do nosso modelo

In [ ]:
ALL_COLUMNS_CONSUMO_GERAL

### Remoção de colunas indesejadas até o momento

In [ ]:
ALL_COLUMNS_CONSUMO_GERAL = ALL_COLUMNS_CONSUMO_GERAL.drop(columns=['Unnamed: 0', 'EMP_CODIGO', 'COD_GRUPO', 'COD_SETOR_COMERCIAL', 'NUM_QUADRA', 'COD_ROTA_LEITURA', 'SEQ_RESPONSAVEL', 'ECO_RESIDENCIAL', 'ECO_COMERCIAL', 'ECO_INDUSTRIAL', 'ECO_PUBLICA', 'ECO_OUTRAS','LTR_ATUAL', 'LTR_COLETADA', 'DAT_LEITURA', 'DIAS_LEITURA', 'COD_LEITURA_INF_1', 'COD_LEITURA_INF_2', 'COD_LEITURA_INF_3', 'HORA_LEITURA', 'DSC_SIMULTANEA', 'COD_LEITURA_INT','EXCECAO'])

In [ ]:
ALL_COLUMNS_CONSUMO_GERAL.columns

### Nessa seção queriamos validar a tabela de "VOLUME_ESTIMADO_ACUM" para ver se ela poderia agregar dentro do nosso modelo

In [ ]:
ALL_COLUMNS_CONSUMO_GERAL[['VOLUME_ESTIMADO_ACUM']].nunique()

In [ ]:
ALL_COLUMNS_CONSUMO_GERAL[ALL_COLUMNS_CONSUMO_GERAL['VOLUME_ESTIMADO'] != 0]

#### O insight retirado aqui é que talvez a melhor coluna para validação e ser utilizada como featura no modelo é a coluna de Volume Estimado. Ela possui maior consistência nos resultados, do que a coluna de Volume Estimado Acumulado

### Separação de Features relevantes

Após pesquisas e visualizar as colunas disponíveis, percebemos uma coluna que poderia ser interessante para o processo de identificação de fraude. A coluna de "DSC_OCORRENCIA". Ela basicamente corresponde a descrição de como foi o processo de vistoria e coleta do responsável e em cada um dos domicílios (matrícula)

Isso surgiu como uma possibilidade de tentar direcionar o modelo para casos nos quais há uma maior possibilidade de uma fraude, de acordo com a visualização do medidor em cada um desses domicílios.

In [ ]:
ALL_COLUMNS_CONSUMO_GERAL_PREMISSA_VINI = ALL_COLUMNS_CONSUMO_GERAL[ALL_COLUMNS_CONSUMO_GERAL['DSC_OCORRENCIA'].isin([
    'NORMAL',
    'MEDIDOR RETIRADO/FURTADO',
    'LEITURA COLETADA PELO CLIENTE',
    'MEDIDOR NÃO LOCALIZADO',
    'IMÓVEL DESOCUPADO'
])]

ALL_COLUMNS_CONSUMO_GERAL_PREMISSA_VINI

## Tratando Dataframe com mais colunas targets adicionadas

#### Seperação do Dataframe apenas para a visualização das matriculas com Categoria Residencial

In [ ]:
dataframe_pj_premissa = ALL_COLUMNS_CONSUMO_GERAL_PREMISSA_VINI[ALL_COLUMNS_CONSUMO_GERAL_PREMISSA_VINI["CATEGORIA"].isin(["COMERCIAL", "PUBLICA", "INDUSTRIAL"])]
dataframe_pj_premissa

### Processo de One Hot Encoding para as Colunas, as quais serão as features para o nosso modelo

In [ ]:
# Fazendo o one hot encoding 

dataframe_pj_premissa = pd.get_dummies(dataframe_pj_premissa, columns=['TIPO_LIGACAO', 'DSC_OCORRENCIA', 'STA_TROCA', 'STA_ACEITA_LEITURA'], dtype='int')

In [ ]:
dataframe_pj_premissa

### Tratando os valores da tabela de fraude

In [ ]:
# analizando as colunas

fraudes.columns

In [ ]:
# removendo os duplicados do dataframe de fraudes
dataframe_fraudes_premissa = fraudes[['MATRICULA', 'DESCRICAO']].drop_duplicates()

# fazendo o one hot encoding do dataframe de fraudes
dataframe_fraudes_premissa = pd.get_dummies(dataframe_fraudes_premissa, columns=['DESCRICAO'], dtype='int')

In [ ]:
# analizando o tipo de dados que temos na tabela de fraudes

dataframe_fraudes_premissa['DESCRICAO_IRREGULARIDADE IDENTIFICADA'].unique()

In [ ]:
# Realiza a junção (merge) entre 'dataframe_pj_premissa' e 'dataframe_fraudes_premissa' com base na coluna 'MATRICULA', 
# mantendo todas as linhas de 'dataframe_pj_premissa' e adicionando informações de 'dataframe_fraudes_premissa' quando disponíveis
dataframe_pj_premissa = pd.merge(dataframe_pj_premissa, dataframe_fraudes_premissa, on='MATRICULA', how='left')

In [ ]:
# dataframe_pj_premissa.drop_duplicates(subset="MATRICULA", keep='first')
dataframe_pj_premissa.dropna(subset="REFERENCIA")

### Normalizando com o Robust Scaler

O RobustScaler é uma técnica de normalização usada para transformar dados. Ele é útil especialmente quando os dados contêm outliers, ou seja, valores atípicos que podem distorcer (neste caso o Consumo e o Volume) os resultados de outras técnicas de escalonamento, como StandardScaler ou MinMaxScaler.

O RobustScaler transforma os dados subtraindo a mediana e dividindo pela amplitude interquartil (IQR, Interquartile Range). A mediana é o valor do ponto médio quando os dados são ordenados, e o IQR é a diferença entre o terceiro quartil (75º percentil) e o primeiro quartil (25º percentil).

Esse método é menos sensível a outliers porque, ao contrário da média e do desvio padrão (usados pelo StandardScaler), a mediana e o IQR não são afetados por valores extremos.

In [ ]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()


# exercendo a normalização
dataframe_pj_premissa[['CONS_MEDIDO']] = scaler.fit_transform(dataframe_pj_premissa[['CONS_MEDIDO']])
dataframe_pj_premissa[['VOLUME_ESTIMADO']] = scaler.fit_transform(dataframe_pj_premissa[['VOLUME_ESTIMADO']])

### Pivotando a tabela de consumo

- Para o index ficar na matrícula e ter uma representação das datas temporais em formato de colunas

In [ ]:
# Cria uma tabela dinâmica (pivot table) a partir de 'dataframe_pj_premissa', usando 'MATRICULA' como índice,
# 'REFERENCIA' como colunas, e somando os valores de 'CONS_MEDIDO' e 'VOLUME_ESTIMADO'
pivoted_df = pd.pivot_table(
    dataframe_pj_premissa,
    index='MATRICULA',
    columns='REFERENCIA',
    values=['CONS_MEDIDO', 'VOLUME_ESTIMADO'],
    aggfunc='sum'
)

# Concatena os nomes das colunas resultantes, unindo os nomes com um underline ('_') e removendo espaços extras
pivoted_df.columns = ['_'.join(col).strip() for col in pivoted_df.columns.values]

# Reseta o índice para que 'MATRICULA' volte a ser uma coluna regular
pivoted_df = pivoted_df.reset_index()

# Exibe as primeiras linhas do DataFrame resultante
pivoted_df.head()

In [ ]:
# analizando resultados

len(pivoted_df)

### Atribuição de cada uma das variáveis categóricas ao dataframe com o consumo e volume históricos

- O código integra dados de tipo de ligação, ocorrências de medidores e identificação de irregularidades ao DataFrame principal, pivoted_df. Após cada junção, os valores nulos são substituídos por zero para garantir a completude dos dados. 

- É importante citar aqui que atribuímos a premissa que a primeira definição de tipo de ligação, descrição e os outros utilizados, serão os que tomaremos como base para inferência dentro do modelo

In [ ]:
# fazendo o tratamento e a junção da coluna de tipo de ligação

tipo_ligacao = dataframe_pj_premissa[['MATRICULA','TIPO_LIGACAO_Consumo Fixo', 'TIPO_LIGACAO_Hidrometrado']].drop_duplicates(subset='MATRICULA', keep='first')
pivoted_df = pivoted_df.merge(tipo_ligacao, on='MATRICULA', how='left').fillna(0)

In [ ]:
# fazendo o tratamento e a junção do tipo de ocorrencia

descricao_ocorrencia = dataframe_pj_premissa[['MATRICULA','DSC_OCORRENCIA_MEDIDOR NÃO LOCALIZADO', 'DSC_OCORRENCIA_MEDIDOR RETIRADO/FURTADO', 'DSC_OCORRENCIA_NORMAL']].drop_duplicates(subset='MATRICULA', keep='first')
pivoted_df = pivoted_df.merge(descricao_ocorrencia, on='MATRICULA', how='left').fillna(0)

In [ ]:
# juntando a tabela de consumo com a de fraude. Preenchendo os valores vazios (as que não estão na tabela de fraude) com zero, representando que eles não cometeram fraude

fraude_ou_não = dataframe_pj_premissa[['MATRICULA','DESCRICAO_IRREGULARIDADE IDENTIFICADA']].drop_duplicates(subset='MATRICULA', keep='first')
pivoted_df = pivoted_df.merge(fraude_ou_não, on='MATRICULA', how='left').fillna(0)

In [ ]:
pivoted_df = pivoted_df.fillna(0)

In [ ]:
pivoted_df

## Rodando o modelo

### Divisão de Treino e Teste

In [ ]:
pivoted_df.shape

#### Balanceando os dados com Smote (Synthetic Minority Over-sampling Technique)

O smote é uma técnica de "oversampling" usada para balancear datasets com o objetivo de aumentar a quantidade de amostras da classe minoritária gerando novos exemplos sintéticos, em vez de simplesmente replicar os dados existentes. Isso ajuda a melhorar a performance de modelos de machine learning ao treinar com um dataset mais balanceado.

Quando você treina um modelo de machine learning com um dataset desbalanceado (onde uma classe tem muito mais exemplos do que a outra), o modelo tende a favorecer a classe majoritária.

Como resultado, o modelo pode ter um bom desempenho em termos de acurácia geral, mas um desempenho ruim ao identificar a classe minoritária (por exemplo, falhas, fraudes, etc.).

O Smote ajuda a mitigar esse problema ao balancear o dataset, aumentando o número de exemplos da classe minoritária.

In [ ]:
# função para balanciar

def balanciar(df):
  smote = SMOTE(random_state=42)
  X = pivoted_df.drop('DESCRICAO_IRREGULARIDADE IDENTIFICADA', axis=1)
  y = pivoted_df['DESCRICAO_IRREGULARIDADE IDENTIFICADA']
  X_res, y_res = smote.fit_resample(X, y)
  return pd.concat([X_res, y_res], axis=1)

In [ ]:
# analizando a distribuição do label nos dados antes do balanciamento

np.unique(pivoted_df['DESCRICAO_IRREGULARIDADE IDENTIFICADA'].values, return_counts=True)

In [ ]:
# balanciando os dodos

pivoted_df = balanciar(pivoted_df.fillna(0))

In [ ]:
# dividindo label e dados para previsão

y = pivoted_df['DESCRICAO_IRREGULARIDADE IDENTIFICADA'].values
X = pivoted_df.drop(['MATRICULA','DESCRICAO_IRREGULARIDADE IDENTIFICADA'], axis=1).values

In [ ]:
# dividindo dados de treinamento e teste

x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
# criando o modelo

import tensorflow.keras as keras
from tensorflow.keras import layers

model = keras.Sequential([
    keras.layers.Input(shape=(x_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# visualizando o modelo

model.summary()

In [ ]:
# compiando o modelo, adicionando metricas para visualizar e a função de perda

model.compile(loss='binary_crossentropy', metrics=['accuracy', 'precision'])

In [ ]:
# Valiando e analizando as dimenções dos dados que irão para o modelo

x_train.shape

In [ ]:
y_train.shape

In [ ]:
x_val.shape

In [ ]:
y_val.shape

In [ ]:
# adicinando early stoping para diminuir o overfit do modelo e parar e fazer um treinamento mais eficiente

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
# treinando o modelo

result = model.fit(x_train, y_train, epochs=25, batch_size=32, validation_data=(x_val, y_val))

### Visualizando os resultados do modelo

In [ ]:
# visão temporal da performance do modelo

history = result.history
fig = px.line(x=list(range(1, 26)), y=history['loss'], labels={'x': 'Épocas', 'y': 'Perda'}, title='Função de Custo durante o Treinamento')
fig.update_traces(mode='lines+markers')
fig.show()

In [ ]:
# matriz de confusão para avaliação da distribuição de erros do modelo

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from keras.models import load_model

y_pred = model.predict(x_val)
y_pred_classes = np.argmax(y_pred, axis=1)

cm = confusion_matrix(y_val, y_pred_classes)

disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.show()

In [ ]:
# gráfico da loss durante o treinamento

plt.plot(result.history['loss'], label='loss')
plt.plot(result.history['val_loss'], label='val_loss')
plt.legend(['Erro Treino', 'Erro Teste'])
plt.xlabel('Épocas')
plt.ylabel('Função de Custo')
plt.title('Training Validation')
plt.show()

In [ ]:
# Análizando a precisão e a perda do modelo

fig = go.Figure()

fig.add_trace(go.Line(x=list(range(1, 26)), y=history['accuracy'], mode='lines+markers', name='Precisão de Treinamento'))

fig.add_trace(go.Line(x=list(range(1, 26)), y=history['val_loss'], mode='lines+markers', name='Perda de Validação'))

fig.update_layout(title='Precisão e Perda durante o Treinamento', xaxis_title='Épocas', yaxis_title='Valor', legend_title='Métrica')
fig.show()

In [ ]:
# ploanto a arquitetura do modelo

from tensorflow.keras.utils import plot_model

plot_model(model, to_file='model_visualization.png', show_shapes=True, show_layer_names=True)